### Importing all the relevant libraries

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
import numpy as np
import sobol_seq
from tabulate import tabulate
import time
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

### Get the estimated expenditure distributions at the member-state level

In [ ]:
df_20072013_ERDF_IT = pd.read_csv('Expenditure_IT.csv',index_col=0)

In [222]:
dist = df_20072013_ERDF_IT.groupby(['NUTS2Code','r'])[[str(y) for y in range(2007,2017)]].sum()

dist.loc['ITH1']=dist.loc['ITH1'].values+dist.loc['ITH2'].values

dist=dist.reset_index()
dist.NUTS2Code[dist.NUTS2Code=='ITH1']='ITH0'
dist = dist[dist.NUTS2Code != 'ITH2']
dist = dist.set_index(['NUTS2Code','r']).sort_index(by=['NUTS2Code','r'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  


In [ ]:
norm_data = pd.read_csv('norm_IT_NUTS2.csv',index_col=[0,1])

### Evaluate their relation

In [ ]:
for row, nuts in dist.groupby(level=0):
    for c in nuts:
        nuts[c].plot.hist(bins = 100, label='estimated expenditure distribution')
        plt.axvspan(norm_data.loc[pd.IndexSlice['min',int(c)],row], norm_data.loc[pd.IndexSlice['max',int(c)],row], alpha=0.25, color='red',
                    label='incurred expenditure range')
        plt.title(row+' '+str(c))
        plt.xlim(min(nuts.min().min(),norm_data[row].min()),max(nuts.max().max(),norm_data[row].max()))
        plt.legend()
        plt.show()